In [248]:
from dotenv import load_dotenv
from pathlib import Path
load_dotenv(Path("./.env"))



True

In [249]:
import os
import requests
import pandas as pd
import json




In [250]:
import requests

# Function to fetch data from the API
def fetch_data(page_size=50, total_rows=350):
    base_url = f"https://developer.nps.gov/api/v1/parks"
    params = {
        'limit': page_size,
        'start': 0,
        'api_key' : '72sxBSZD8hwyJmyjjELBZ6Rt6xwfYdMUizYrMegg'
    }
    all_data = []

    # Calculate number of requests needed
    num_requests = (total_rows + page_size - 1) // page_size

    for page in range(num_requests):
        params['start'] = page * page_size
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            all_data.extend(response.json()['data'])
        else:
            print(f"Failed to fetch data for page {page + 1}")

    return all_data

# Fetch 200 rows of data
data = fetch_data(total_rows=350)

# Now 'data' contains all the rows fetched from the API
df_Parks = pd.DataFrame(data)
df_Parks

,id,url,fullName,parkCode,description,latitude,longitude,latLong,activities,topics,...,fees,directionsInfo,directionsUrl,operatingHours,addresses,images,weatherInfo,name,designation,relevanceScore
0,77E0D7F0-1942-494A-ACE2-9004D2BDC59E,https://www.nps.gov/abli/index.htm,Abraham Lincoln Birthplace National Historical...,abli,For over a century people from around the worl...,37.5858662,-85.67330523,"lat:37.5858662, long:-85.67330523",[{'id': '13A57703-BB1A-41A2-94B8-53B692EB7238'...,[{'id': 'D10852A3-443C-4743-A5FA-6DD6D2A054B3'...,...,[],The Birthplace Unit of the park is located app...,http://www.nps.gov/abli/planyourvisit/directio...,"[{'exceptions': [{'exceptionHours': {}, 'start...","[{'postalCode': '42748', 'city': 'Hodgenville'...","[{'credit': 'NPS Photo', 'title': 'The Memoria...",There are four distinct seasons in Central Ken...,Abraham Lincoln Birthplace,National Historical Park,1.0
1,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,https://www.nps.gov/acad/index.htm,Acadia National Park,acad,Acadia National Park protects the natural beau...,44.409286,-68.247501,"lat:44.409286, long:-68.247501",[{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42'...,[{'id': '00F3C3F9-2D67-4802-81AE-CCEA5D3BA370'...,...,[],"From Boston take I-95 north to Augusta, Maine,...",http://www.nps.gov/acad/planyourvisit/directio...,"[{'exceptions': [], 'description': 'Acadia Nat...","[{'postalCode': '04609', 'city': 'Bar Harbor',...","[{'credit': 'NPS / Kristi Rugg', 'title': 'Aca...","Located on Mount Desert Island in Maine, Acadi...",Acadia,National Park,1.0
2,E4C7784E-66A0-4D44-87D0-3E072F5FEF43,https://www.nps.gov/adam/index.htm,Adams National Historical Park,adam,From the sweet little farm at the foot of Penn...,42.2553961,-71.01160356,"lat:42.2553961, long:-71.01160356",[{'id': 'B33DC9B6-0B7D-4322-BAD7-A13A34C584A3'...,[{'id': 'F3883A66-A7CB-461B-868E-1B5932224B25'...,...,[],"Traveling on U.S. Interstate 93, take exit 7 -...",http://www.nps.gov/adam/planyourvisit/directio...,[{'exceptions': [{'exceptionHours': {'wednesda...,"[{'postalCode': '02169', 'city': 'Quincy', 'st...","[{'credit': 'NPS Photo', 'title': 'The John an...","Be prepared for hot, humid weather. The histor...",Adams,National Historical Park,1.0
3,1A47416F-DAA3-4137-9F30-14AF86B4E547,https://www.nps.gov/afam/index.htm,African American Civil War Memorial,afam,"Over 200,000 African-American soldiers and sai...",38.9166,-77.026,"lat:38.9166, long:-77.026",[{'id': 'B33DC9B6-0B7D-4322-BAD7-A13A34C584A3'...,[{'id': '28AEAE85-9DDA-45B6-981B-1CFCDCC61E14'...,...,[],The memorial is located at the corner of Vermo...,http://www.nps.gov/afam/planyourvisit/directio...,"[{'exceptions': [], 'description': 'The Africa...","[{'postalCode': '20001', 'city': 'Washington',...","[{'credit': 'NPS Photo', 'title': 'African Ame...",Washington DC gets to see all four seasons. Hu...,African American Civil War Memorial,,1.0
4,E6E1D22A-7A89-47F8-813C-B611059A8CF9,https://www.nps.gov/afbg/index.htm,African Burial Ground National Monument,afbg,African Burial Ground is the oldest and larges...,40.71452681,-74.00447358,"lat:40.71452681, long:-74.00447358",[{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42'...,[{'id': '28AEAE85-9DDA-45B6-981B-1CFCDCC61E14'...,...,[],The African Burial Ground National Monument is...,http://www.nps.gov/afbg/planyourvisit/directio...,[{'exceptions': [{'exceptionHours': {'monday':...,"[{'postalCode': '10007', 'city': 'New York', '...","[{'credit': 'NPS Photo', 'title': 'African Bur...",http://forecast.weather.gov/MapClick.php?CityN...,African Burial Ground,National Monument,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,6282ABFA-00EF-413D-BE91-48D993A356F4,https://www.nps.gov/paav/index.htm,Pennsylvania Avenue,paav,A street unlike any other. It is known the wor...,38.89426856,-77.02506554,"lat:38.89426856, long:-77.02506554",[{'id': 'B33DC9B6-0B7D-4322-BAD7-A13A34C584A3'...,[{'id': 'F669BC40-BDC4-41C0-9ACE-B2CD25373045'...,...,[],Pennsylvania Avenue 

In [253]:
def merge_dictionary(lst):
    merged_dictnry = {}
    for d in lst:
        merged_dictnry.update(d)
    return merged_dictnry

# Apply the function to each row of the "address" column
# df_Parks['addresses'] = df_Parks['addresses'].apply(merge_dictionary)
df_Parks['images'] = df_Parks['images'].apply(merge_dictionary)




In [255]:
def extract_contact_info(row):
    phone_numbers = row.get('phoneNumbers', [])
    email_addresses = row.get('emailAddresses', [])
    
    phone_number = phone_numbers[0]['phoneNumber'] if phone_numbers else ''
    email_address = email_addresses[0]['emailAddress'] if email_addresses else ''
    
    return {'phoneNumber': phone_number, 'emailAddress': email_address}


# Apply the function to the 'column_name' column
df_Parks['extracted_contact'] = df_Parks['contacts'].apply(extract_contact_info)
df_Parks['extracted_contact']

0      {'phoneNumber': '2703583137', 'emailAddress': ...
1      {'phoneNumber': '2072883338', 'emailAddress': ...
2      {'phoneNumber': '617-773-1177', 'emailAddress'...
3      {'phoneNumber': '2024266841', 'emailAddress': ...
4      {'phoneNumber': '2122384367', 'emailAddress': ...
                             ...                        
345    {'phoneNumber': '2024266841', 'emailAddress': ...
346    {'phoneNumber': '4192852184', 'emailAddress': ...
347    {'phoneNumber': '8047323531', 'emailAddress': ...
348    {'phoneNumber': '9285246228', 'emailAddress': ...
349    {'phoneNumber': '5058990205', 'emailAddress': ...
Name: extracted_contact, Length: 350, dtype: object

In [252]:
df_Parks['addresses']

0      {'postalCode': '42748', 'city': 'Hodgenville',...
1      {'postalCode': '04609', 'city': 'Bar Harbor', ...
2      {'postalCode': '02169', 'city': 'Quincy', 'sta...
3      {'postalCode': '20242', 'city': 'Washington', ...
4      {'postalCode': '10005', 'city': 'New York', 's...
                             ...                        
345    {'postalCode': '20242', 'city': 'Washington', ...
346    {'postalCode': '43456', 'city': 'Put-in-Bay', ...
347    {'postalCode': '23803', 'city': 'Petersburg', ...
348    {'postalCode': '86028-2217', 'city': 'Petrifie...
349    {'postalCode': '87120', 'city': 'Albuquerque',...
Name: addresses, Length: 350, dtype: object

In [254]:
df_parks['images']

0     {'credit': 'NPS Photo', 'title': 'Statue of th...
1     {'credit': 'NPS / Kristi Rugg', 'title': 'Clim...
2     {'credit': 'NPS Photo', 'title': 'The Gardens ...
3     {'credit': 'NPS / Liz Macro', 'title': 'Africa...
4     {'credit': 'NPS Photo', 'title': 'Exhibits wit...
5     {'credit': 'NPS Photo', 'title': 'The James H....
6     {'credit': 'NPS Photo', 'title': 'On the histo...
7     {'credit': 'NPS', 'title': 'Alagnak Wild River...
8     {'credit': 'NPS/Josh Spice', 'title': 'Fairban...
9     {'credit': 'Darlyne Sheppard Alcatraz Photo Co...
10    {'credit': 'Photo courtesy National Archives',...
11    {'credit': 'NPS Photo', 'title': 'Snowy Alibat...
12    {'credit': 'NPS', 'title': 'Barrels and crates...
13    {'credit': 'NPS Photo / Stuart West', 'title':...
14    {'credit': 'NPS photo', 'title': 'The beach at...
15    {'credit': 'Photo by: Octavio Garcia', 'title'...
16    {'credit': 'NPS', 'title': 'Bring your dog to ...
17    {'credit': 'NPS Photo', 'title': 'National

In [259]:
df_phone_email = df_Parks['extracted_contact'].apply(pd.Series)
df_phone_email.rename(columns={c:'park_' + c for c in df_phone_email.columns},
                 inplace=True)

df_phone_email


,park_phoneNumber,park_emailAddress
0,2703583137,ABLI_Administration@nps.gov
1,2072883338,acadia_information@nps.gov
2,617-773-1177,ADAM_Visitor_Center@nps.gov
3,2024266841,national_mall@nps.gov
4,2122384367,african_burial_ground@nps.gov
...,...,...
345,2024266841,national_mall@nps.gov
346,4192852184,pevi_social_media@nps.gov
347,8047323531,Pete_Interpretation@nps.gov
348,9285246228,PEFO_information@nps.gov


In [257]:
df_address = df_Parks['addresses'].apply(pd.Series)
df_address.rename(columns={c:'address_' + c for c in df_address.columns},
                 inplace=True)

df_address


,address_postalCode,address_city,address_stateCode,address_countryCode,address_provinceTerritoryCode,address_line1,address_type,address_line3,address_line2
0,42748,Hodgenville,KY,US,,2995 Lincoln Farm Road,Mailing,,
1,04609,Bar Harbor,ME,US,,PO Box 177,Mailing,,
2,02169,Quincy,MA,US,,135 Adams Street,Mailing,,
3,20242,Washington,DC,US,,1100 Ohio Drive SW,Mailing,,
4,10005,New York,NY,US,,African Burial Ground NM,Mailing,26 Wall St,C/O Federal Hall National Memorial
...,...,...,...,...,...,...,...,...,...
345,20242,Washington,DC,US,,1100 Ohio Drive SW,Mailing,,
346,43456,Put-in-Bay,OH,US,,PO Box 549,Mailing,,
347,23803,Petersburg,VA,US,,Petersburg National Battlefield Administration...,Mailing,,1539 Hickory Hill Road
348,86028-2217,Petrified Forest,AZ,US,,Petrified Forest National Park,Mailing,Attn.:,P.O. Box 2217


In [258]:
df_images = df_Parks['images'].apply(pd.Series)
df_images.rename(columns={c:'images_' + c for c in df_images.columns},
                 inplace=True)

df_images

,images_credit,images_title,images_altText,images_caption,images_url
0,NPS Photo,Statue of the Lincoln Family in the Visitor Ce...,Statue of the Lincoln family in the park's Vis...,Visitors to the park can view the statue of th...,https://www.nps.gov/common/uploads/structured_...
1,NPS / Kristi Rugg,Climbing The Precipice,Two hikers ascend a sheer cliff trail by way o...,Whether it's a stroll along Ocean Path or a di...,https://www.nps.gov/common/uploads/structured_...
2,NPS Photo,The Gardens Located at Old House at Peace field,The gardens located at Old House at Peace field.,The gardens located by Old House at Peace fiel...,https://www.nps.gov/common/uploads/structured_...
3,NPS / Liz Macro,African American Civil War Memorial,Bronze statue of African American Civil War so...,African American Civil War Memorial,https://www.nps.gov/common/uploads/structured_...
4,NPS Photo,Exhibits within visitor center.,Exhibits within visitor center.,Exhibits within visitor center.,https://www.nps.gov/common/uploads/structured_...
...,...,...,...,...,...
345,NPS / Chris Elbich,Stephenson Memorial (Grand Army of the Republic),"A rather squat, stone obelisk in a plaza with ...",The Grand Army of the Republic was a veterans ...,https://www.nps.gov/common/uploads/structured_...
346,NPS 2023 photo Mimi Gorman,Visitor center porch and window,"A tan building with a 2-story center window, p...",Just as welcoming as the front door the back s...,https://www.nps.gov/common/uploads/structured_...
347,NPS,Fort Fisher,Earthworks covered in green grass.,Earthworks at Fort Fisher,https://www.nps.gov/common/uploads/structured_...
348,NPS/Hallie Larsen,Tandem Bicycling at Blue Mesa,Two people in colorful clothing on a tandem bi...,Tandem Bicycling at Blue Mesa,https://www.nps.gov/common/uploads/structured_...


Merging all the dataframes together into df_Parks

In [260]:
df_Parks = pd.concat((df_Parks,df_address,df_images,df_phone_email), axis=1)
df_Parks

,id,url,fullName,parkCode,description,latitude,longitude,latLong,activities,topics,...,address_type,address_line3,address_line2,images_credit,images_title,images_altText,images_caption,images_url,park_phoneNumber,park_emailAddress
0,77E0D7F0-1942-494A-ACE2-9004D2BDC59E,https://www.nps.gov/abli/index.htm,Abraham Lincoln Birthplace National Historical...,abli,For over a century people from around the worl...,37.5858662,-85.67330523,"lat:37.5858662, long:-85.67330523",[{'id': '13A57703-BB1A-41A2-94B8-53B692EB7238'...,[{'id': 'D10852A3-443C-4743-A5FA-6DD6D2A054B3'...,...,Mailing,,,NPS Photo,Statue of the Lincoln Family in the Visitor Ce...,Statue of the Lincoln family in the park's Vis...,Visitors to the park can view the statue of th...,https://www.nps.gov/common/uploads/structured_...,2703583137,ABLI_Administration@nps.gov
1,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,https://www.nps.gov/acad/index.htm,Acadia National Park,acad,Acadia National Park protects the natural beau...,44.409286,-68.247501,"lat:44.409286, long:-68.247501",[{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42'...,[{'id': '00F3C3F9-2D67-4802-81AE-CCEA5D3BA370'...,...,Mailing,,,NPS / Kristi Rugg,Climbing The Precipice,Two hikers ascend a sheer cliff trail by way o...,Whether it's a stroll along Ocean Path or a di...,https://www.nps.gov/common/uploads/structured_...,2072883338,acadia_information@nps.gov
2,E4C7784E-66A0-4D44-87D0-3E072F5FEF43,https://www.nps.gov/adam/index.htm,Adams National Historical Park,adam,From the sweet little farm at the foot of Penn...,42.2553961,-71.01160356,"lat:42.2553961, long:-71.01160356",[{'id': 'B33DC9B6-0B7D-4322-BAD7-A13A34C584A3'...,[{'id': 'F3883A66-A7CB-461B-868E-1B5932224B25'...,...,Mailing,,,NPS Photo,The Gardens Located at Old House at Peace field,The gardens located at Old House at Peace field.,The gardens located by Old House at Peace fiel...,https://www.nps.gov/common/uploads/structured_...,617-773-1177,ADAM_Visitor_Center@nps.gov
3,1A47416F-DAA3-4137-9F30-14AF86B4E547,https://www.nps.gov/afam/index.htm,African American Civil War Memorial,afam,"Over 200,000 African-American soldiers and sai...",38.9166,-77.026,"lat:38.9166, long:-77.026",[{'id': 'B33DC9B6-0B7D-4322-BAD7-A13A34C584A3'...,[{'id': '28AEAE85-9DDA-45B6-981B-1CFCDCC61E14'...,...,Mailing,,,NPS / Liz Macro,African American Civil War Memorial,Bronze statue of African American Civil War so...,African American Civil War Memorial,https://www.nps.gov/common/uploads/structured_...,2024266841,national_mall@nps.gov
4,E6E1D22A-7A89-47F8-813C-B611059A8CF9,https://www.nps.gov/afbg/index.htm,African Burial Ground National Monument,afbg,African Burial Ground is the oldest and larges...,40.71452681,-74.00447358,"lat:40.71452681, long:-74.00447358",[{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42'...,[{'id': '28AEAE85-9DDA-45B6-981B-1CFCDCC61E14'...,...,Mailing,26 Wall St,C/O Federal Hall National Memorial,NPS Photo,Exhibits within visitor center.,Exhibits within visitor center.,Exhibits within visitor center.,https://www.nps.gov/common/uploads/structured_...,2122384367,african_burial_ground@nps.gov
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,6282ABFA-00EF-413D-BE91-48D993A356F4,https://www.nps.gov/paav/index.htm,Pennsylvania Avenue,paav,A street unlike any other. It is known the wor...,38.89426856,-77.02506554,"lat:38.89426856, long:-77.02506554",[{'id': 'B33DC9B6-0B7D-4322-BAD7-A13A34C584A3'...,[{'id': 'F669BC40-BDC4-41C0-9ACE-B2CD25373045'...,...,Mailing,,,NPS / Chris Elbich,Stephenson Memorial (Grand Army of the Republic),"A rather squat, stone obelisk in a plaza with ...",The Grand Army of the Republic was a veterans ...,https://www.nps.gov/common/uploads/structured_...,2024266841,national_mall@nps.gov
346,29329926-4215-4E10-83B8-5CF162A0E3F1,https://www.nps.gov/pevi/index.htm,Perry's Victory & International Peace Memorial,pevi,Perry's Victory and International Peace Memori...,41.65449782,-82.8115794,"lat:41.65449782, long:-82.8115794",

Columns in the dataset

In [281]:
df_Parks.columns

Index(['id', 'url', 'fullName', 'parkCode', 'description', 'latitude',
       'longitude', 'latLong', 'states', 'directionsInfo', 'directionsUrl',
       'weatherInfo', 'name', 'designation', 'relevanceScore',
       'address_postalCode', 'address_city', 'address_stateCode',
       'address_countryCode', 'address_provinceTerritoryCode', 'address_line1',
       'address_type', 'address_line3', 'address_line2', 'images_credit',
       'images_title', 'images_altText', 'images_caption', 'images_url',
       'park_phoneNumber', 'park_emailAddress'],
      dtype='object')

Saving it as CSV file

In [282]:
df_Parks.to_csv('/Users/sneha/IUPUI-INFORMATICS/INFO501-LAB/Nationalpark-project/NPS.ipynbnational_parks.csv')

Below is the cleaned data containing 350 rows of data related to national park

In [283]:
df_Parks

,id,url,fullName,parkCode,description,latitude,longitude,latLong,states,directionsInfo,...,address_type,address_line3,address_line2,images_credit,images_title,images_altText,images_caption,images_url,park_phoneNumber,park_emailAddress
0,77E0D7F0-1942-494A-ACE2-9004D2BDC59E,https://www.nps.gov/abli/index.htm,Abraham Lincoln Birthplace National Historical...,abli,For over a century people from around the worl...,37.5858662,-85.67330523,"lat:37.5858662, long:-85.67330523",KY,The Birthplace Unit of the park is located app...,...,Mailing,,,NPS Photo,Statue of the Lincoln Family in the Visitor Ce...,Statue of the Lincoln family in the park's Vis...,Visitors to the park can view the statue of th...,https://www.nps.gov/common/uploads/structured_...,2703583137,ABLI_Administration@nps.gov
1,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,https://www.nps.gov/acad/index.htm,Acadia National Park,acad,Acadia National Park protects the natural beau...,44.409286,-68.247501,"lat:44.409286, long:-68.247501",ME,"From Boston take I-95 north to Augusta, Maine,...",...,Mailing,,,NPS / Kristi Rugg,Climbing The Precipice,Two hikers ascend a sheer cliff trail by way o...,Whether it's a stroll along Ocean Path or a di...,https://www.nps.gov/common/uploads/structured_...,2072883338,acadia_information@nps.gov
2,E4C7784E-66A0-4D44-87D0-3E072F5FEF43,https://www.nps.gov/adam/index.htm,Adams National Historical Park,adam,From the sweet little farm at the foot of Penn...,42.2553961,-71.01160356,"lat:42.2553961, long:-71.01160356",MA,"Traveling on U.S. Interstate 93, take exit 7 -...",...,Mailing,,,NPS Photo,The Gardens Located at Old House at Peace field,The gardens located at Old House at Peace field.,The gardens located by Old House at Peace fiel...,https://www.nps.gov/common/uploads/structured_...,617-773-1177,ADAM_Visitor_Center@nps.gov
3,1A47416F-DAA3-4137-9F30-14AF86B4E547,https://www.nps.gov/afam/index.htm,African American Civil War Memorial,afam,"Over 200,000 African-American soldiers and sai...",38.9166,-77.026,"lat:38.9166, long:-77.026",DC,The memorial is located at the corner of Vermo...,...,Mailing,,,NPS / Liz Macro,African American Civil War Memorial,Bronze statue of African American Civil War so...,African American Civil War Memorial,https://www.nps.gov/common/uploads/structured_...,2024266841,national_mall@nps.gov
4,E6E1D22A-7A89-47F8-813C-B611059A8CF9,https://www.nps.gov/afbg/index.htm,African Burial Ground National Monument,afbg,African Burial Ground is the oldest and larges...,40.71452681,-74.00447358,"lat:40.71452681, long:-74.00447358",NY,The African Burial Ground National Monument is...,...,Mailing,26 Wall St,C/O Federal Hall National Memorial,NPS Photo,Exhibits within visitor center.,Exhibits within visitor center.,Exhibits within visitor center.,https://www.nps.gov/common/uploads/structured_...,2122384367,african_burial_ground@nps.gov
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,6282ABFA-00EF-413D-BE91-48D993A356F4,https://www.nps.gov/paav/index.htm,Pennsylvania Avenue,paav,A street unlike any other. It is known the wor...,38.89426856,-77.02506554,"lat:38.89426856, long:-77.02506554",DC,Pennsylvania Avenue National Historic Site is ...,...,Mailing,,,NPS / Chris Elbich,Stephenson Memorial (Grand Army of the Republic),"A rather squat, stone obelisk in a plaza with ...",The Grand Army of the Republic was a veterans ...,https://www.nps.gov/common/uploads/structured_...,2024266841,national_mall@nps.gov
346,29329926-4215-4E10-83B8-5CF162A0E3F1,https://www.nps.gov/pevi/index.htm,Perry's Victory & International Peace Memorial,pevi,Perry's Victory and International Peace Memori...,41.65449782,-82.8115794,"lat:41.65449782, long:-82.8115794",OH,Perry's Victory and International Peace Memori...,...,Mailing,,,NPS 2023 photo Mimi Gorman,Visitor center porch and window,"A tan building with a 2-story center window, p...",Just as welcoming as the front door the back s...,https://www.nps.gov/common/uploads/structured_